## Import Libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader, TensorDataset, random_split

## Data Preparation

In [4]:
df=pd.read_csv("cars-85.csv")
df.dropna(inplace=True)
df.shape

(193, 24)

In [5]:
for i in df.columns:
    if df[i].dtypes=="object":
        df[i] = df[i].astype('category')
        df[i] =df[i].cat.codes

In [6]:
targets = df[["price"]].to_numpy()
inputs = df.drop(["price"], axis=1).to_numpy()  

In [7]:
inputs = torch.from_numpy(inputs)
inputs = inputs.float()
targets = torch.from_numpy(targets)
targets = targets.float()

In [8]:
inputs_mean = torch.mean(inputs, axis=0)
targets_mean = torch.mean(targets, axis=0)
inputs = inputs / inputs_mean  
targets = targets / targets_mean 

In [9]:
dataset = TensorDataset(inputs, targets)

In [10]:
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

## Define Two Models

In [11]:
model = nn.Sequential(nn.Linear(23, 16),
                      nn.Sigmoid(),
                       
                      nn.Linear(16, 8),
                      nn.Softmax(),
                       
                       
                      nn.Linear(8, 4),
                      nn.ReLU(),

                      
                      nn.Linear(4, 1),
                     nn.ReLU())
model_with_optim = nn.Sequential(nn.Linear(23, 16),
                      nn.Dropout(p=0.2),
                       
                      nn.Linear(16, 8),
                      nn.Softmax(),
                       
                       
                      nn.Linear(8, 4),
                      nn.ReLU(),

                      
                      nn.Linear(4, 1),
                     nn.ReLU())



In [12]:
# Utility function to train the model
def fit(num_epochs, model, criterion, optimizer, train_loader):
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        # Train with batches of data
        for xb,yb in train_loader:
            # 1. Generate predictions
            pred = model(xb)
            # 2. Calculate loss
            loss = criterion(pred, yb)
            # 3. Compute gradients
            loss.backward()
            # 4. Update parameters using gradients
            optimizer.step()
            # 5. Reset the gradients to zero
            optimizer.zero_grad()
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

## Fit Model

In [13]:
criterion = F.mse_loss
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
fit(200, model, criterion, optimizer, train_loader)

C:\Users\user\Anaconda3\lib\site-packages\torch\nn\modules\container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [10/200], Loss: 0.3225
Epoch [20/200], Loss: 0.7181
Epoch [30/200], Loss: 4.3910
Epoch [40/200], Loss: 0.3971
Epoch [50/200], Loss: 0.3488
Epoch [60/200], Loss: 0.7045
Epoch [70/200], Loss: 0.2810
Epoch [80/200], Loss: 0.2651
Epoch [90/200], Loss: 0.6312
Epoch [100/200], Loss: 0.3029
Epoch [110/200], Loss: 1.2351
Epoch [120/200], Loss: 0.8397
Epoch [130/200], Loss: 0.7980
Epoch [140/200], Loss: 2.1441
Epoch [150/200], Loss: 0.1124
Epoch [160/200], Loss: 1.4914
Epoch [170/200], Loss: 0.2081
Epoch [180/200], Loss: 0.3680
Epoch [190/200], Loss: 0.1573
Epoch [200/200], Loss: 1.6273


## Fit Model with regularization

In [14]:
criterion = F.mse_loss
optimizer = torch.optim.SGD(model.parameters(),weight_decay=2, lr=1e-4)
fit(200, model, criterion, optimizer, train_loader)

Epoch [10/200], Loss: 0.4449
Epoch [20/200], Loss: 6.2941
Epoch [30/200], Loss: 0.1693
Epoch [40/200], Loss: 0.6991
Epoch [50/200], Loss: 0.0545
Epoch [60/200], Loss: 0.4971
Epoch [70/200], Loss: 0.0441
Epoch [80/200], Loss: 5.0617
Epoch [90/200], Loss: 1.1865
Epoch [100/200], Loss: 1.0677
Epoch [110/200], Loss: 1.0560
Epoch [120/200], Loss: 0.2489
Epoch [130/200], Loss: 0.3382
Epoch [140/200], Loss: 0.0344
Epoch [150/200], Loss: 0.2710
Epoch [160/200], Loss: 1.4175
Epoch [170/200], Loss: 0.8608
Epoch [180/200], Loss: 0.0793
Epoch [190/200], Loss: 0.1356
Epoch [200/200], Loss: 0.2496


## Fit Model with regularization and dropout

In [16]:
criterion = F.mse_loss
optimizer = torch.optim.SGD(model_with_optim.parameters(), lr=1e-4)
fit(200, model_with_optim, criterion, optimizer, train_loader)

Epoch [10/200], Loss: 0.6793
Epoch [20/200], Loss: 0.4781
Epoch [30/200], Loss: 0.2111
Epoch [40/200], Loss: 1.1140
Epoch [50/200], Loss: 1.5051
Epoch [60/200], Loss: 0.3059
Epoch [70/200], Loss: 0.1523
Epoch [80/200], Loss: 0.0750
Epoch [90/200], Loss: 0.6349
Epoch [100/200], Loss: 0.2739
Epoch [110/200], Loss: 1.0196
Epoch [120/200], Loss: 0.7099
Epoch [130/200], Loss: 0.0485
Epoch [140/200], Loss: 1.0911
Epoch [150/200], Loss: 0.8562
Epoch [160/200], Loss: 5.8013
Epoch [170/200], Loss: 0.9585
Epoch [180/200], Loss: 0.2295
Epoch [190/200], Loss: 0.3142
Epoch [200/200], Loss: 0.4066
